In [75]:
# Import libraries
import os
import requests
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from tensorflow_metadata.proto.v0 import schema_pb2
from tfx.v1.components import ImportSchemaGen
from tfx.types import artifact_utils
from tfx.types.standard_artifacts import Schema
import tensorflow_transform as tft
from tfx.components import Transform

print('TF version:', tf.__version__)
print('TFDV version:', tfdv.version.__version__)


TF version: 2.13.0
TFDV version: 1.14.0


# 2. Descripción del Conjunto de Datos

In [5]:
# Directory of the raw data files
_data_root = './data/covertype'
# Path to the raw training data
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')

In [3]:
## download the dataset

# Download data
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
    #https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
    url = 'https://docs.google.com/uc?export= \
    download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
    r = requests.get(url, allow_redirects=True, stream=True)
    open(_data_filepath, 'wb').write(r.content)

In [4]:
data = pd.read_csv(_data_filepath)

In [5]:
data.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,119,7,67,11,1015,233,234,133,1570,Commanche,C7202,1
1,2876,3,18,485,71,2495,192,202,144,1557,Commanche,C7757,1
2,3171,315,2,277,9,4374,213,237,162,1052,Rawah,C7745,0
3,3087,342,13,190,31,4774,193,221,166,752,Rawah,C7745,0
4,2835,158,10,212,41,3596,231,242,141,3280,Rawah,C4744,1


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116203 entries, 0 to 116202
Data columns (total 13 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Elevation                           116203 non-null  int64 
 1   Aspect                              116203 non-null  int64 
 2   Slope                               116203 non-null  int64 
 3   Horizontal_Distance_To_Hydrology    116203 non-null  int64 
 4   Vertical_Distance_To_Hydrology      116203 non-null  int64 
 5   Horizontal_Distance_To_Roadways     116203 non-null  int64 
 6   Hillshade_9am                       116203 non-null  int64 
 7   Hillshade_Noon                      116203 non-null  int64 
 8   Hillshade_3pm                       116203 non-null  int64 
 9   Horizontal_Distance_To_Fire_Points  116203 non-null  int64 
 10  Wilderness_Area                     116203 non-null  object
 11  Soil_Type                           116

## Inicializar el repositorio de DVC y añadir los datos

In [8]:
#!dvc init

In [8]:
!git commit -m 'Initialize DVC'

[main 06c1f44] Initialize DVC
 3 files changed, 6 insertions(+)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .dvcignore


In [9]:
!dvc add data/

⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/work/.dvc/cache/files/md5'| |0/? [00:00<?,    ?files/s]
                                                                                
!
  0%|          |Adding data to cache                  0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /work/data               0/2 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00, 24.26file/s]

To track the changes with git, run:

	git add .gitignore data.dvc

To enable auto staging, run:

	dvc config core.autostage true


# 3. Seleción de características

In [9]:
# Selecciona solo las características numéricas
data_numeric_features = data.select_dtypes(include=['number'])
# Separar target de features
X = data_numeric_features.drop('Cover_Type', axis=1)  # Assuming 'Cover_Type' is the target
y = data['Cover_Type']

# Feature selection
selector = SelectKBest(f_classif, k=8) 
X_new = selector.fit_transform(X, y)

selected_features = X.columns[selector.get_support()]
features_retain = {feature: (feature in selected_features) for feature in X.columns}

In [10]:
features_retain

{'Elevation': True,
 'Aspect': False,
 'Slope': True,
 'Horizontal_Distance_To_Hydrology': True,
 'Vertical_Distance_To_Hydrology': True,
 'Horizontal_Distance_To_Roadways': True,
 'Hillshade_9am': True,
 'Hillshade_Noon': True,
 'Hillshade_3pm': False,
 'Horizontal_Distance_To_Fire_Points': True}

In [25]:
data_selected_features = data[list(selected_features) + 
                              ['Wilderness_Area', 'Soil_Type', 'Cover_Type']]

In [26]:
data_selected_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116203 entries, 0 to 116202
Data columns (total 11 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Elevation                           116203 non-null  int64 
 1   Slope                               116203 non-null  int64 
 2   Horizontal_Distance_To_Hydrology    116203 non-null  int64 
 3   Vertical_Distance_To_Hydrology      116203 non-null  int64 
 4   Horizontal_Distance_To_Roadways     116203 non-null  int64 
 5   Hillshade_9am                       116203 non-null  int64 
 6   Hillshade_Noon                      116203 non-null  int64 
 7   Horizontal_Distance_To_Fire_Points  116203 non-null  int64 
 8   Wilderness_Area                     116203 non-null  object
 9   Soil_Type                           116203 non-null  object
 10  Cover_Type                          116203 non-null  int64 
dtypes: int64(9), object(2)
memory usage: 9.

In [27]:
data_selected_features.to_csv(_data_filepath,index=False)

# 4. Data Pipeline

## 4.1 Configurar el contexto interactivo

In [2]:
import tensorflow as tf

from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

import os
import pprint
pp = pprint.PrettyPrinter()

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6Status12empty_stringB5cxx11Ev']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZNK10tensorflow4data11DatasetBase8FinalizeEPNS_15OpKernelContextESt8functionIF

In [3]:
_pipeline_root = './pipeline/'
context = InteractiveContext(pipeline_root=_pipeline_root)

## 4.2 Generando Ejemplos Con CsvExampleGen

In [6]:
example_gen = CsvExampleGen(input_base = _data_root)

In [7]:
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 7
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## 4.3 Estadísticas

In [8]:
# Crear una instancia de StatisticsGen con ExampleGen
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

#Ejecutar el componente
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 8
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [9]:
context.show(statistics_gen.outputs['statistics'])

## 4.4 Inferir el esquema 

In [10]:
schema_gen = SchemaGen(statistics = statistics_gen.outputs['statistics'],)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 9
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [11]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,-
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


## 4.5 Curando el esquema

In [12]:
schema_path = schema_gen.outputs['schema']._artifacts[0].uri + '/schema.pbtxt'

In [13]:
schema = tfdv.load_schema_text(schema_path)

In [17]:
tfdv.set_domain(schema, 'Hillshade_9am', schema_pb2.IntDomain(name='Hillshade_9am', min=0, max=255))
tfdv.set_domain(schema, 'Hillshade_Noon', schema_pb2.IntDomain(name='Hillshade_Noon', min=0, max=255))
tfdv.set_domain(schema, 'Slope', schema_pb2.IntDomain(name='Slope', min=0, max=90))

In [18]:
cover_type_feature = tfdv.get_feature(schema, 'Cover_Type')
cover_type_feature.int_domain.is_categorical = True

In [19]:
cover_type_feature.int_domain.min = 0
cover_type_feature.int_domain.max = 6

In [20]:
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


In [21]:
cover_type_feature.int_domain.is_categorical

True

## 4.6 Entornos de esquema

In [22]:
# Todas las características están en el entorno de entrenamiento
schema.default_environment.append('TRAINING')
schema.default_environment.append('SERVING')

# Sacar 
tfdv.get_feature(schema, 'Cover_Type').not_in_environment.append('SERVING')

In [23]:
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


In [41]:
# Verificar los entornos definidos en el esquema
print('Entornos definidos en el esquema:', schema.default_environment)

# Verificar la configuración específica de entorno para la característica 'Cover_Type'
cover_type_feature = tfdv.get_feature(schema, 'Cover_Type')
if cover_type_feature:
    print('Entornos excluidos para Cover_Type:', cover_type_feature.not_in_environment)
else:
    print('La característica Cover_Type no se encuentra en el esquema.')

Entornos definidos en el esquema: ['TRAINING', 'SERVING']
Entornos excluidos para Cover_Type: ['SERVING']


In [42]:
# Guardar el esquema modificado
schema_path = os.path.join(_pipeline_root, 'schema.pptxt')
tfdv.write_schema_text(schema, schema_path)

In [43]:
df = pd.read_csv(_data_filepath)

# Eliminar la columna de etiqueta para simular datos de servicio
SERVING_DATA = df.drop('Cover_Type', axis=1)

Para verificar el correcto funcionamiento de los diferentes entornos, se realiza la validación para los dos 

In [45]:
serving_statistics = tfdv.generate_statistics_from_dataframe(SERVING_DATA)
anomalies = tfdv.validate_statistics(statistics=serving_statistics, schema=schema, environment='SERVING')
tfdv.display_anomalies(anomalies)

In [46]:
anomalies = tfdv.validate_statistics(statistics=serving_statistics, schema=schema, environment='TRAINING')
tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',Column dropped,Column is completely missing


## 4.7 Genere nuevas estadı́sticas usando el esquema actualizado

In [68]:
#Crear un nuevo artefacto que apunte al esquema anterior guardado

new_schema = ImportSchemaGen(schema_file=schema_path)
context.run(new_schema)

ExecutionResult(
    component_id: ImportSchemaGen
    execution_id: 17
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=ImportSchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [60]:
new_statistics_gen = StatisticsGen(
    examples = example_gen.outputs['examples'],
    schema = new_schema.outputs['schema']
)
context.run(new_statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 14
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [59]:
context.show(new_statistics_gen.outputs['statistics'])

Se puede observar que la columna Cover_type, está correctamente configurada como una característica categórica. Así como las "Hillshade_9am", "Hillshade_Noon", "Slope", se encuentran dentro de los rangos aceptados. 

## 4.8 Comprobar anomalı́as

In [69]:
example_validator = ExampleValidator(
    statistics = new_statistics_gen.outputs['statistics'],
    schema = new_schema.outputs['schema']
)
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 18
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [70]:
context.show(example_validator.outputs['anomalies'])

```Se observa que no existen anomalias y que está correcatamente configurado el esquema nuevo```

## 4.9 Ingenierı́a de caracterı́sticas

## 4.10 Funciones de procesamiento

- Las características numéricas se decide escalarlas con z score
- **Soil Type** se decide procesar con hash_strings teniendo en cuenta que la cantidad de categorías se podría considerar alta (40)
- **Wilderness_Area** se procesa con vocabulario dado que no tiene una gran cantidad de categorías 
- La etiqueta no se transforma

In [74]:
def preprocessing_fn(inputs):
    outputs = {}
    
    # Escalar características numéricas con z score
    for feature in ['Elevation', 'Slope', 'Horizontal_Distance_To_Hydrology',
                    'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
                    'Hillshade_9am', 'Hillshade_Noon', 'Horizontal_Distance_To_Fire_Points']:
        outputs[feature] = tft.scale_to_z_score(inputs[feature])
    
    outputs['Soil_Type'] = tft.hash_strings(inputs['Soil_Type'], hash_bucket=80)
    
    # Convertir características categóricas en índices de vocabulario
    outputs['Wilderness_Area'] = tft.compute_and_apply_vocabulary(inputs['Wilderness_Area'])
    # No transformar la etiqueta
    outputs['Cover_Type'] = inputs['Cover_Type']
    
    return outputs

## 4.11. Transformar

In [82]:
import preprocessing
transform = Transform(
    examples = example_gen.outputs['examples'],
    schema = new_schema.outputs['schema'],
    preprocessing_fn = 'preprocessing.preprocessing_fn'
)

context.run(transform)

INFO:tensorflow:Assets written to: ./pipeline/Transform/transform_graph/23/.temp_path/tftransform_tmp/54381d42b3ea49faa33d7bd1cbf4b883/assets


INFO:tensorflow:Assets written to: ./pipeline/Transform/transform_graph/23/.temp_path/tftransform_tmp/54381d42b3ea49faa33d7bd1cbf4b883/assets


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 23
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [101]:
context.show(transform.outputs['post_transform_stats'])

# 5. Metadatos de aprendizaje automático

In [102]:
from ml_metadata import metadata_store
from ml_metadata.proto import metadata_store_pb2

# Configura la conexión al almacén de metadatos
connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.sqlite.filename_uri = "./pipeline/metadata.sqlite"
connection_config.sqlite.connection_mode = 3  # READWRITE_OPENCREATE

store = metadata_store.MetadataStore(connection_config)

## 5.1 Acceso a artefactos almacenados

In [103]:
artifact_types = store.get_artifact_types()
for artifact_type in artifact_types:
    print(artifact_type.name)


Examples
ExampleStatistics
Schema
ExampleAnomalies
TransformGraph
TransformCache


Aquí se pueden observar todos los esquemas almacenados en la base de datos, como se puede observar existen multiples esquemas. sin embargo el esque con Id 6 y 9 son los inferidos por SchemaGen

In [117]:
schema_type_name = 'Schema'

schemas = store.get_artifacts_by_type(schema_type_name)

# Muestra información sobre los esquemas
for schema in schemas:  
    print(f"ID del Esquema: {schema.id}")
    print("Propiedades:")
    for key, value in schema.custom_properties.items(): 
        if value.HasField('string_value'):
            print(f"  {key}: {value.string_value}")
        elif value.HasField('int_value'):  
            print(f"  {key}: {value.int_value}")
    print("----------")


ID del Esquema: 6
Propiedades:
  tfx_version: 1.14.0
  name: schema:2024-03-03T14:32:00.267721
  producer_component: SchemaGen
----------
ID del Esquema: 9
Propiedades:
  tfx_version: 1.14.0
  producer_component: SchemaGen
  name: schema:2024-03-03T22:07:27.767892
----------
ID del Esquema: 10
Propiedades:
  name: schema:2024-03-04T01:54:54.449227
  producer_component: ImportSchemaGen
  tfx_version: 1.14.0
----------
ID del Esquema: 18
Propiedades:
  name: pre_transform_schema:2024-03-04T02:59:40.535471
  producer_component: Transform
----------
ID del Esquema: 20
Propiedades:
  producer_component: Transform
  name: post_transform_schema:2024-03-04T02:59:40.535471
----------
ID del Esquema: 26
Propiedades:
  name: pre_transform_schema:2024-03-04T03:01:03.035517
  producer_component: Transform
----------
ID del Esquema: 28
Propiedades:
  producer_component: Transform
  name: post_transform_schema:2024-03-04T03:01:03.035517
----------
ID del Esquema: 34
Propiedades:
  name: pre_transform

In [118]:
schema_type_name = 'ExampleStatistics'

schemas = store.get_artifacts_by_type(schema_type_name)

# Muestra información sobre los esquemas
for schema in schemas:  
    print(f"ID del Esquema: {schema.id}")
    print("Propiedades:")
    for key, value in schema.custom_properties.items(): 
        if value.HasField('string_value'):
            print(f"  {key}: {value.string_value}")
        elif value.HasField('int_value'):  
            print(f"  {key}: {value.int_value}")
    print("----------")

ID del Esquema: 3
Propiedades:
  tfx_version: 1.14.0
  producer_component: StatisticsGen
  name: statistics:2024-03-03T14:22:22.611136
  stats_dashboard_link: 
----------
ID del Esquema: 5
Propiedades:
  producer_component: StatisticsGen
  tfx_version: 1.14.0
  stats_dashboard_link: 
  name: statistics:2024-03-03T14:26:53.203957
----------
ID del Esquema: 8
Propiedades:
  producer_component: StatisticsGen
  stats_dashboard_link: 
  tfx_version: 1.14.0
  name: statistics:2024-03-03T22:07:10.194662
----------
ID del Esquema: 11
Propiedades:
  stats_dashboard_link: 
  tfx_version: 1.14.0
  producer_component: StatisticsGen
  name: statistics:2024-03-04T02:07:39.225880
----------
ID del Esquema: 19
Propiedades:
  name: pre_transform_stats:2024-03-04T02:59:40.535471
  producer_component: Transform
----------
ID del Esquema: 21
Propiedades:
  name: post_transform_stats:2024-03-04T02:59:40.535471
  producer_component: Transform
----------
ID del Esquema: 27
Propiedades:
  producer_component: 

In [121]:
schema_type_name = 'TransformGraph'

schemas = store.get_artifacts_by_type(schema_type_name)

# Muestra información sobre los esquemas
for schema in schemas:  
    print(f"ID del Esquema: {schema.id}")
    print("Propiedades:")
    for key, value in schema.custom_properties.items(): 
        if value.HasField('string_value'):
            print(f"  {key}: {value.string_value}")
        elif value.HasField('int_value'):  
            print(f"  {key}: {value.int_value}")
    print("----------")

ID del Esquema: 15
Propiedades:
  producer_component: Transform
  name: transform_graph:2024-03-04T02:59:40.535471
----------
ID del Esquema: 23
Propiedades:
  name: transform_graph:2024-03-04T03:01:03.035517
  producer_component: Transform
----------
ID del Esquema: 31
Propiedades:
  name: transform_graph:2024-03-04T03:01:37.647546
  producer_component: Transform
----------
ID del Esquema: 39
Propiedades:
  producer_component: Transform
  name: transform_graph:2024-03-04T03:01:58.044130
----------
ID del Esquema: 47
Propiedades:
  producer_component: Transform
  tfx_version: 1.14.0
  name: transform_graph:2024-03-04T03:06:12.325011
----------


## 5.2 Seguimiento de artefactos

In [152]:
def find_input_artifacts(store, artifact_id):
    """
    Encuentra los artefactos de entrada que llevaron a la creación de un artefacto específico.
    
    Args:
    store: Una instancia de metadata_store.MetadataStore.
    artifact_id: El ID del artefacto del cual se desean encontrar los artefactos de entrada.
    
    Returns:
    Una lista de artefactos que son entradas para el artefacto dado.
    """
    # Encuentra todos los eventos relacionados con el artefacto dado
    events = store.get_events_by_artifact_ids([artifact_id])
    
    input_artifacts = []
    for event in events:

        # Encuentra las ejecuciones relacionadas con este evento
        execution_id = event.execution_id
        # Encuentra los eventos de esta ejecución 
        related_events = store.get_events_by_execution_ids([execution_id])
        for related_event in related_events:
            if related_event.type == metadata_store_pb2.Event.OUTPUT:
                # Encuentra y añade el artefacto de salida (entrada para el artefacto dado)
                artifact = store.get_artifacts_by_id([related_event.artifact_id])[0]
                input_artifacts.append(artifact)
    
    return input_artifacts

In [153]:
input_artifacts = find_input_artifacts(store,1)
for artifact in input_artifacts:
    print(f"ID del Artefacto: {artifact.id}, Tipo: {artifact.type_id}")

ID del Artefacto: transform_graph:2024-03-04T03:06:12.325011, Tipo: 23
ID del Artefacto: transformed_examples:2024-03-04T03:06:12.325011, Tipo: 14
ID del Artefacto: updated_analyzer_cache:2024-03-04T03:06:12.325011, Tipo: 24
ID del Artefacto: pre_transform_schema:2024-03-04T03:06:12.325011, Tipo: 18
ID del Artefacto: pre_transform_stats:2024-03-04T03:06:12.325011, Tipo: 16
ID del Artefacto: post_transform_schema:2024-03-04T03:06:12.325011, Tipo: 18
ID del Artefacto: post_transform_stats:2024-03-04T03:06:12.325011, Tipo: 16
ID del Artefacto: post_transform_anomalies:2024-03-04T03:06:12.325011, Tipo: 21


In [159]:
def track_artifact_inputs(store, artifact_id):
    """
    Rastrea los artefactos de entrada que condujeron a la creación de un artefacto específico.
    
    Args:
    store: Una instancia de metadata_store.MetadataStore para interactuar con MLMD.
    artifact_id: El ID del artefacto para el cual rastrear las entradas.
    
    Returns:
    Un diccionario con los IDs de artefacto de entrada y sus tipos y URIs correspondientes.
    """
    # Obtén los eventos asociados con el artefacto dado
    events = store.get_events_by_artifact_ids([artifact_id])
    
    input_artifacts_info = {}
    
    for event in events:
    # Busca ejecuciones donde el artefacto dado fue una salida
        execution_id = event.execution_id
        input_events = store.get_events_by_execution_ids([execution_id])

        for input_event in input_events:
            # Identifica los artefactos de entrada en la ejecución
            input_artifact_id = input_event.artifact_id
            input_artifact = store.get_artifacts_by_id([input_artifact_id])[0]
            # Guarda la información relevante del artefacto de entrada
            input_artifacts_info[input_artifact_id] = {
                'type': store.get_artifact_types_by_id([input_artifact.type_id])[0].name,
                'uri': input_artifact.uri
            }
    
    return input_artifacts_info


In [161]:
# Primer artefacto
artifact_id = 1
input_artifacts_info = track_artifact_inputs(store, artifact_id)

for id, info in input_artifacts_info.items():
    print(f"Artifact ID: {id}, Type: {info['type']}, URI: {info['uri']}")

Artifact ID: 1, Type: Examples, URI: ./pipeline/CsvExampleGen/examples/1


In [162]:
# cuarto artefacto
artifact_id = 4
input_artifacts_info = track_artifact_inputs(store, artifact_id)

for id, info in input_artifacts_info.items():
    print(f"Artifact ID: {id}, Type: {info['type']}, URI: {info['uri']}")

Artifact ID: 4, Type: Examples, URI: ./pipeline/CsvExampleGen/examples/4
Artifact ID: 5, Type: ExampleStatistics, URI: ./pipeline/StatisticsGen/statistics/5
